<a href="https://colab.research.google.com/github/mobinapourmoshir/Functional-Deep-Learning/blob/main/DNN%20Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Libraries
from sklearn.datasets import load_iris
import pandas as pd
import torch

In [2]:
# Load the Iris dataset
iris = load_iris()
print(iris.keys())

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])


The loaded dataset is a Bunch object (scikit-learn datasets), which is similar to a dictionary. It contains:

- data: The feature matrix (sepal length, sepal width, petal length, petal width)
- target: The target labels (species of iris flower: 0, 1, 2)
- feature_names: Names of the features
- target_names: Names of the target classes (setosa, versicolor, virginica)
- DESCR: A description of the dataset

To convert it to a data frame using pandas:

In [3]:
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
print(df.head())

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.1               3.5                1.4               0.2
1                4.9               3.0                1.4               0.2
2                4.7               3.2                1.3               0.2
3                4.6               3.1                1.5               0.2
4                5.0               3.6                1.4               0.2


## Difference between Tensor data set and data loader

**Tensor:** just multi-dimensional arrays, like an extension of lists in Python.

**TensorDataset:** Assume features (input data) and labels (answers). TensorDataset pairs them together so you can access each input with its label.

**DataLoader:** Feeds your data in batches to a model so you don’t have to handle one sample at a time. Can also shuffle the data for training.